# Slots

The attributes of objects are stored in a dictionary `__dict__`. Like any other dictionary, a dictionary used for attribute storage doesn't have a fixed number of elements. In other words, you can add elements to dictionaries after they have been defined.

This is the reason, we can dynamically add attributes to objects of classes that we have created.


In [1]:
>>> class Foo(object):
...     pass

In [2]:
a = Foo()

In [3]:
a.x = 128

In [5]:
a.y = "Dynamically created Attribute!"

In [6]:
a.__dict__

{'x': 128, 'y': 'Dynamically created Attribute!'}

In [7]:
import pprint
pprint.pprint(a.__class__.__dict__)

mappingproxy({'__dict__': <attribute '__dict__' of 'Foo' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Foo' objects>})


## We can't do this with built-in classes like **`int`**, or **`list`**.

In [8]:
a = int()

In [9]:
a.x = 5

AttributeError: 'int' object has no attribute 'x'

In [10]:
a = dict()

In [11]:
a.y = 5

AttributeError: 'dict' object has no attribute 'y'

Using a dictionary for attribute storage is very convenient, but it can mean a waste of space for objects, which have only a small amount of instance variables. The space consumption can become critical when creating large numbers of instances. Slots are a nice way to work around this space consumption problem. Instead of having a dynamic dict that allows adding attributes to objects dynamically, slots provide a static structure which prohibits additions after the creation of an instance.

When we design a class, we can use slots to prevent the dynamic creation of attributes. To define slots, you have to define a list with the name `__slots__`. The list has to contain all the attributes, you want to use. We demonstrate this in the following class, in which the slots list contains only the name for an attribute `val`.

In [12]:
>>> class Foo(object):
...     __slots__ = ['val']
...     def __init__(self, v):
...         self.val = v

In [13]:
x = Foo(42)

In [14]:
x.val = 100
x.val

100

In [15]:
x.new = "not possible"

AttributeError: 'Foo' object has no attribute 'new'

# Relationship between Class and type

In [16]:
x = [4, 5, 9]

In [17]:
y = "Hello"

In [18]:
print(type(x), type(y))

<class 'list'> <class 'str'>


If you apply type on the name of a class itself, you get the class **`type`** returned.

In [19]:
print(type(list), type(str)) # This is similar to applying type on type(x) and type(y)

<class 'type'> <class 'type'>


In [20]:
print(type(type(x)), type(type(y)))

<class 'type'> <class 'type'>


# Classes and Class Creation

`type(classname, superclasses, attributes_dict)`

In [21]:
>>> class Foo(object):
...     pass

In [22]:
a = Foo()

In [23]:
type(a)

__main__.Foo

In [24]:
type(type(a))

type

In [25]:
Foo = type("Foo", (), {})

In [26]:
x = Foo()

In [27]:
print(type(x))

<class '__main__.Foo'>


When we call **`type`**, the call method of type is called. The call method runs two other methods: **`new`** and **`init`**
```
type.__new__(typeclass, classname, superclasses, attributedict)
type.__init__(cls, classname, superclasses, attributedict)
```


**Class creating without **

In [28]:
>>> class ObjectWithOutType(object):
...     counter = 0
...     def __init__(self, name):
...         self.name = name
...     def about(self):
...         return 'This is test hello!' + self.name

In [29]:
y = ObjectWithOutType('without')

In [30]:
print(y.name)

without


In [31]:
print(y.about())

This is test hello!without


In [32]:
print(y.__dict__)

{'name': 'without'}


The **new** method creates and returns the **new** class object, and after this the **new** method initializes the newly created object.

In [33]:
>>> def object_init(self, name):
...     self.name = name
>>> ObjectWithType = type('ObjectWithType',
...                       (),
...                       {'counter': 0,
...                       '__init__': object_init,
...                       'about': lambda self: 'This is from Object with type! ' + self.name})

In [34]:
x = ObjectWithType('with')

In [35]:
print(x.name)

with


In [39]:
print(x.__dict__)

{'name': 'with'}


In [40]:
import pprint
pprint.pprint(x.__class__.__dict__)
pprint.pprint(y.__class__.__dict__)

mappingproxy({'__dict__': <attribute '__dict__' of 'ObjectWithType' objects>,
              '__doc__': None,
              '__init__': <function object_init at 0x1064321e0>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'ObjectWithType' objects>,
              'about': <function <lambda> at 0x106432158>,
              'counter': 0})
mappingproxy({'__dict__': <attribute '__dict__' of 'ObjectWithOutType' objects>,
              '__doc__': None,
              '__init__': <function ObjectWithOutType.__init__ at 0x106432048>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'ObjectWithOutType' objects>,
              'about': <function ObjectWithOutType.about at 0x1064320d0>,
              'counter': 0})


# More metaclasses

In [41]:
>>> class BaseClass1:
...     def ret_method(self, *args):
...         return 5

>>> class BaseClass2:
...     def ret_method(self, *args):
...         return 5

In [42]:
a = BaseClass1()
a.ret_method()

5

In [43]:
b = BaseClass2()
b.ret_method()

5

**Another way to achieve this**

In [44]:
>>> class SuperClass(object):
...     def ret_method(self, *args):
...         return 5

>>> class BaseClass1(SuperClass):
...     pass

>>> class BaseClass2(SuperClass):
...     pass

In [45]:
a = BaseClass1()
a.ret_method()

5

In [46]:
b = BaseClass2()
b.ret_method()

5

Let's assume, we don't know a previously if we want or need this method. Let's assume that the decision, if the classes have to be augmented, can only be made at runtime. This decision might depend on configuration files, user input or some calculations.

In [47]:
required = True

In [48]:
>>> def ret_method(self, *args):
...     return 5

In [49]:
>>> class BaseClass1(object):
...     pass
>>> if required:
...     BaseClass1.ret_method = ret_method
... class BaseClass2(object):
...     pass
>>> if required:
...     BaseClass2.ret_method = ret_method

In [50]:
a = BaseClass1()
a.ret_method()

5

In [51]:
b = BaseClass2()
b.ret_method()

5

We have to add the same code to every class and it seems likely that we might forget it. Hard to manage and maybe even confusing, if there are many methods we want to add.

In [52]:
required = True

In [54]:
def ret_method(self, *args):
    return 5
def augment_method(cls):
    if required:
        cls.ret_method = ret_method
    return cls
class BaseClass1(object):
    pass
augment_method(BaseClass1)
class BaseClass2(object):
    pass
augment_method(BaseClass2)

__main__.BaseClass2

In [55]:
a = BaseClass1()
a.ret_method()

5

In [56]:
b = BaseClass2()
b.ret_method()

5

This is good but we need to make sure the code is executed.


The code should be executed **automatically**. We need a way to make sure that "some" code might be executed automatically after the end of a class definition.

In [57]:
required = True

In [59]:
>>> def ret_method(self, *args):
...     return 5
>>> def augment_method(cls):
...     if required:
...         print('Adding')
...         cls.ret_method = ret_method
...     return cls
>>> @augment_method
... class BaseClass1:
...     pass
>>> @augment_method
... class BaseClass2:
...     pass
# BaseClass1 = augment_method(BaseClass1)
# BaseClass2 = augment_method(BaseClass2)

Adding
Adding


In [60]:
a = BaseClass1()
a.ret_method()

5

In [61]:
b = BaseClass2()
b.ret_method()

5

# Singleton

In [62]:
>>> class Singleton(type):
...     _instances = {}
...     def __call__(cls, *args, **kwargs):
...         if cls not in cls._instances:
...             cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
...         return cls._instances[cls]

>>> class SingletonClass(metaclass=Singleton):
...     pass

>>> class RegularClass():
...     pass

In [63]:
x = SingletonClass()

In [64]:
y = SingletonClass()

In [65]:
print(x == y)

True


In [66]:
x = RegularClass()

In [67]:
y = RegularClass()

In [68]:
print(x == y)

False


In [ ]:
print(id(x), id()